In this notebook, we convert a CellDesigner map into an executable qualitative model available as SBML-qual and GINsim files.

First, let's select a cell designer map.

In [1]:
# List xml files in the current folder
!ls *.xml

IFN_26042020.xml  Interferon_corrected_26_04_20.xml


In [2]:
# Select the one we will use in this notebook (without the xml extension)
model_name = "IFN_26042020"

# Create an executable model with CaSQ¶


In [3]:
import casq.celldesigner2qual as casq

# debug messages are enabled by default
casq.logger.disable("casq")


# Load and simplify a cell designer map
info, width, height = casq.read_celldesigner(f"{model_name}.xml")
casq.simplify_model(info)

# Write the SBML file
casq.write_qual(f"{model_name}.sbml", info, width, height)

# Load and view the model in GINsim¶

In [4]:
import biolqm
import ginsim


# Load the model in bioLQM
m = biolqm.load(f"{model_name}.sbml")

# Transfer component names to identifiers
m = biolqm.sanitize(m)

# Adjust the layout
layout = m.getLayout().scale(0.5, 0.3).crop()

# Transfer to GINsim
lrg = biolqm.to_ginsim(m)
ginsim.show(lrg)

In [5]:
# Save this model in GINsim format
lrg.save(f"{model_name}.zginml")